# **Cvičení 3: Jak se správně trefit do terče**
---


V tomto cvičení si ukážeme malou ukázku optimalizačních úloh. Ukážeme si metodu nejmenších čtverců a její využití při výpočtu optimálního míření pro trefení do terče.

---

*Toto jsou knihovny a funkcionality, které budeme využívat:*

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import numba

*Toto jsou funkce, které jsme vyrobili v předchozím cvičení a budeme je využívat pro simulace střelby na terč:*

In [27]:
@numba.jit
def model_pohybu(beta, gamma, rho, alpha, wx, wy):
    # vstupni parametry jsou:
    # beta - úhel vychýlení oprotí míření na střed v ose z
    # gamma - úhel vychýlení oprotí míření na střed v ose y
    # rho - ustova rychlost
    # alpha - koeficient odporu vzduchu
    # wx - rychlost větru ve směru osy x
    # wy - rychlost větru ve směru osy y
    # výstupní hodnoty jsou:
    # x, y - souřadnice kde střela dopadne na rovině terče

    # inicializace proměnných
    g = 9.81
    m = 0.00051
    dt = 0.01
    n_max = 100
    vzdalenost_terce = 50
    r0 = np.array([0, 0, 0])

    # výpočet ůhlů pro transformaci počáteční rychlosti
    phi = np.pi/2 - np.deg2rad(beta)
    theta = np.deg2rad(gamma)

    # výpočet počáteční rychlosti
    v0 = rho * np.array([np.sin(phi)*np.cos(theta), np.sin(phi)*np.sin(theta), np.cos(phi)])

    # inicializace polí pro trajektorii, rychlost a zrychlení
    r = np.zeros((n_max, 3))
    v = np.zeros((n_max, 3))
    a = np.zeros((n_max, 3))
    
    # inicializace vektorů gravitace a větru
    g_vec = np.array([0, 0, -g])
    w_vec = np.array([wx, wy, 0])

    # počáteční podmínky
    r[0] = r0
    v[0] = v0
    
    for i in range(1, n_max):
        a[i - 1] = g_vec - alpha / m * np.linalg.norm(v[i - 1] - w_vec) * (v[i - 1] - w_vec)
        v[i] = v[i - 1] + a[i - 1] * dt
        r[i] = r[i - 1] + (v[i - 1] + v[i]) / 2 * dt
        if r[i, 0] > vzdalenost_terce:  # pozici počítáme rovnou a při zásahu terče ukončíme cyklus
            break
    
    # poslední dvě hodnoty jsou právě těsně před a po zásahu terče
    j = i - 1
    y = r[j, 1] + (r[i, 1] - r[j, 1]) / (r[i, 0] - r[j, 0]) * (vzdalenost_terce - r[j, 0])
    z = r[j, 2] + (r[i, 2] - r[j, 2]) / (r[i, 0] - r[j, 0]) * (vzdalenost_terce - r[j, 0])
    return z, y


def vystrel(beta, gamma):
    # skryté parametry, pozor nekoukat!!!
    return model_pohybu(beta, gamma, 200, 1.7e-5, -5, 3)

---

## Střelec matematik

Matematik si koupil vzduchovku a chce se naučit střílet. Jelikož má za to, že si to umí spočítat, jde rovnou střílet na 50 metrů vzdálený terč.


### První čtyři střely

Na rozehřátí se rozhodl vystřelit první čtyři střely. A zaznamenal si jak úhly náklonu zbraňe (ve stupních) tak pozice zásahu na "terči" (ano, v prvních třech případech to byla zeď na které byl terč pověšen, ale změřit to šlo) v metrech.

In [28]:
# úhly hlavně ve stupních oproti výstřelu přímo na terč
beta = np.array([1, 2.5, 1.5, 1.8])  # úhel ve stupních, náklon v ose z (kladné jsou nahoru)
gamma = np.array([-1, -2, -1.5, -1.7]) # úhel ve stupních, náklon v ose y (kladné jsou doprava)

# naměřené souřadnice zásahu terče v metrech
z = np.array([-0.91936, 0.44877, -0.46353, -0.18994]) # souřadnice zásahu v ose z v metrech
y = np.array([ 0.60353, -0.30889, 0.14654,-0.03597]) # souřadnice zásahu v ose y v metrech

To matematikovi stačilo. Vzal tušku a papír něco si tam načmáral, spokojeně pokýval hlavou, vzal puškou a vystřelil. 

Přesně na střed... ani ne milimetr od středu.

Nespokojeně zabrblal a šel domů. Tam si to naťukal do počítače a výsledek opsal na papír...

Druhý den se vrátil a vystřelil jednu střelu. Dokonalý střed, tak přesně se to ještě nikomu nepovedlo!

Spokojeně se usmál, prodal vzduchovku s tím, že už to má vyřešené a šel domů.

**Zvládneme to i my?**

---

# Popis Úlohy
Našim úkolem je určit ideální úhly pro trefení cíle v bodě $(0, 0)$ na terči. K dispozici máme množinu $N$ měření, kde každé měření obsahuje dvojici vstupních úhlů (úhly vychýlení oproti míření na střed terčy v $z$ a $y$ ose) $(\beta, \gamma)$ a odpovídající souřadnice zásahu $(z, y)$.

**Jak takovou úlohu vyřešit?**

- Hledáme nějakou kombinaci (funkci) vstupních parametrů $\beta$ a $\gamma$, která nám umožní predikovat zásah do terče.
- Pokud takovou funkci najdeme, můžeme ji využít k určení ideálních úhlů pro zásah do bodu $(0, 0)$.

**Jakou funci použít?**


## Formulace Lineárního Modelu

- Takovým nejzákladnějším modelem je tzv. lineární model. Je to navíc rozumná volba neboť máme velice málo měření (pokusů).
- Lineární model znamená, že výsledek závisí lineárně na vstupních parametrech. Například tato funkce je lineární: $$f(x) = a \cdot x + b.$$ V našem případě máme parametry dva a také dva výstupy, budeme mít tedy dva lineární modely:
$$
\begin{aligned}
    z &= a_{1} \cdot \beta + a_{2} \cdot \gamma + a_3, \\
    y &= b_{1} \cdot \beta + b_{2} \cdot \gamma + b_3.
\end{aligned}
$$
Zde $a_{1}, a_{2}, a_3, b_{1}, b_{2}, b_3$ jsou parametry modelu, které budeme určovat.

**Jak určit parametry modelu?**

## Metoda Nejmenších Čtverců
Pro určení parametrů použijeme metodu nejmenších čtverců (Známe z přednášky!). Naším cílem je minimalizovat chybu mezi skutečnými měřeními a hodnotami predikovanými modelem. Jelikož máme modely dva, budeme určovat dvě sady parametrů 
$$
\mathbf{a} = \begin{bmatrix} a_1 \\ a_2 \\ a_3 \end{bmatrix} \text{ a } \mathbf{b} = \begin{bmatrix} b_1 \\ b_2 \\ b_3 \end{bmatrix}
$$.

### Krok 1: Matematická Formulace
Pro $N$ měření zapíšeme data ve vektorovém tvaru:

$$
\mathbf{M} \mathbf{a} = \mathbf{z}, \quad \mathbf{M} \mathbf{b} = \mathbf{y},
$$

kde:
- $\mathbf{z}$ a $\mathbf{y}$ jsou vektory naměřených pozic zásahů $z$ a $y$, 
- $\mathbf{M}$ je matice vytvořená z naměřených hodnot tak ať pomocí její aplikace získáme predikce modelu, tedy obsahuje vstupní úhly a konstantní člen: $$a_{1} \cdot \beta + a_{2} \cdot \gamma + a_3 \cdot 1$$ $$b_{1} \cdot \beta + b_{2} \cdot \gamma + b_3 \cdot 1$$ tedy matice $\mathbf{M}$ má tvar:
$$
\mathbf{M} = \begin{bmatrix}
\beta_1 & \gamma_1 & 1 \\
\beta_2 & \gamma_2 & 1 \\
\vdots & \vdots & \vdots \\
\beta_N & \gamma_N & 1
\end{bmatrix},
$$

Z přednášky víme, ža naše soustavy rovnic 
$$
\mathbf{M} \mathbf{a} = \mathbf{z}, \quad \mathbf{M} \mathbf{b} = \mathbf{y},
$$
nemusí mít řešení. Dle metody nejmenších čtverců tedy hledáme takové $\mathbf{a}$ a $\mathbf{b}$, které minimalizují chybu mezi predikcemi modelu a skutečnými měřeními. A z přednášky víme, že takové $\mathbf{a}$ a $\mathbf{b}$ lze najít jako řešení soustav rovnic:
$$
\mathbf{M}^T \mathbf{M} \mathbf{a} = \mathbf{M}^T \mathbf{z}, \quad \mathbf{M}^T \mathbf{M} \mathbf{b} = \mathbf{M}^T \mathbf{y}.
$$

---
## Ukol 1.

Vytvořte matici $\mathbf{M}$ a a použijte měřaní zásahů jako vektory $\mathbf{z}$ a $\mathbf{y}$.

Spočtěte soustavy rovnic pro nejměnší čtverce a určete parametry $\mathbf{a}$ a $\mathbf{b}$.

In [ ]:
# Ukol 1

######## doplnte kod #######################

# sestavení matice M
N = len(z)
M = np.vstack((beta, gamma, np.ones(N))).T

# výpočet a
a = np.linalg.solve(M.T @ M, M.T @ z)

# výpočet b
b = np.linalg.solve(M.T @ M, M.T @ y)

##############################################

print('a =', a, 'b =', b)

---

## Jak pomocí modelu určit ideální úhly pro trefení do terče?

Nyní když máme model, můžeme ho použít k určení ideálních úhlů pro trefení do terče:
- chceme trefit bod $(0, 0)$, tedy $z = 0$ a $y = 0$
- toto, pro náš lineární model, odpovídá soustavě rovnic:
$$
\begin{aligned}
    0 &= a_{1} \cdot \beta + a_{2} \cdot \gamma + a_3, \\
    0 &= b_{1} \cdot \beta + b_{2} \cdot \gamma + b_3.
\end{aligned}
$$
- maticově lze tuto soustavu zapsat jako:
$$ \mathbf{A} \mathbf{x} = \mathbf{f},$$
kde: 
$$
\mathbf{A} = \begin{bmatrix}
a_{1} & a_{2} \\
b_{1} & b_{2}
\end{bmatrix},\quad \mathbf{x} = \begin{bmatrix} \beta \\ \gamma \end{bmatrix}, \quad \mathbf{f} = \begin{bmatrix} -a_3 \\ -b_3 \end{bmatrix}.
$$

---
## Ukol 2.

Sestavte matice $\mathbf{A}$ a $\mathbf{f}$ a určete ideální úhly pro trefení do terče.



In [ ]:
# Ukol 2

######## doplnte kod #######################
A = np.array([[a[0], a[1]], [b[0], b[1]]])
f = np.array([-a[2], -b[2]])

x = np.linalg.solve(A, f)
##############################################
print(f"Optimalni uhly pro strelbu jsou beta = {x[0]:.3f} [°], gamma = {x[1]:.3f} [°]")

A ještě zkontrolujeme kam se trefíme....

In [ ]:
# ověříme, kam střela dopadne
z_trefa, y_trefa = vystrel(x[0], x[1])
print(f"Střela dopadne na souřadnice z = {1000 * z_trefa:.2f} [mm], y = {1000 * y_trefa:.2f} [mm]")

**Jsme tedy na milimetr přesně ve středu terče!**

---

## Lze to udělat lépe?

Asi ano, když se tak ptám...

**Využijeme znalost modelu z minulého cvičení!**

Výše je přepsána funkce, která modeluje zásahy do terče. 

Volat se dá takto: `model_pohybu(beta, rho, gamma, alpha, wx, wy)`, kde:
- `beta`, `gamma` jsou úhly vychýlení střely od osy $z$ a $y$,
- `rho` je úsťová rychlost střely,
- `alpha` je koeficient odporu vzduchu,
- `wx`, `wy` jsou rychlosti větru ve směru os $z$ a $y$.

A vrací nám vektor `[z, y]` s pozicí zásahu.

Ostatní parametry (již součástí modelu) jsou známé nebo jednoduše ověřitelné bez střelby:
- hmotnost střely $m = 0.00051$ [$kg$],
- gravitační zrychlení $g = 9.81$ [$m/s^2$],
- vzdálenost terče $50$ [$m$].

## Co je nyní náš model?

- S modelem je to nyní komplikovanější, je to poměrně složitá nelineární funkce (celá naše simulace trajektorie).
- Ale víme jaké jsou naše parametry modelu: $\rho$, $\alpha$, $w_x$, $w_y$.
- Náš model si tedy můžeme představit jako parametrizovanou funkci, tedy funkci, která v závoslosti na parametrech $\rho$, $\alpha$, $w_x$, $w_y$ počítá souřadnice zásahu do terče pro vstupní úhly $\beta$ a $\gamma$:
$$ \mathbf{f}(\beta, \gamma; \rho, \alpha, w_x, w_y) = \begin{bmatrix} z \\ y \end{bmatrix}.$$

- Můžeme použít zobecněnou metodu nejmenších čtverců, tedy pro všechna naměřená data spočítat kvadráty odchylky mezi naměřenými hodnotami a hodnotami predikovanými modelem a minimalizovat je. Chyba bude tedy: $$\mathbf{E}(\rho, \alpha, w_x, w_y) = \sum_{i=1}^{N} \left\Vert \mathbf{f}(\beta_i, \gamma_i; \rho, \alpha, w_x, w_y) - \begin{bmatrix} z_i \\ y_i \end{bmatrix} \right\Vert^2,$$ 
kde $\left\Vert \cdot \right\Vert^2$ je Euklidovská norma vektoru na druhou. Optimální parametry našeho modelu jsou pak ty které mají minimalní chybu $\mathbf{E}$ :
$$
\rho^*, \alpha^*, w_x^*, w_y^* = \arg \min_{\rho, \alpha, w_x, w_y} \mathbf{E}(\rho, \alpha, w_x, w_y).
$$


**Nelekejme se!**

Problém sice vypadá abstraktně a komplikovaně, ale to nám ve skutečnosti nevadí, protože existují úžasné matematické knihovny, které nám pomohou s výpočtem. 

Například operaci $\arg \min_{\rho, \alpha, w_x, w_y}$ nám zvládne spočítat funkce `minimize` z knihovny `scipy.optimize`. Budeme akorát muset vyrobit funkci počítající chybu $\mathbf{E}$. A zadat počáteční odhad parametrů.

---
## Ukol 3.

Sestavte funkci počítající chybu $\mathbf{E}$ pro naše naměřená data a použíjte ji ve funkci `minimize` k nalezení optimálních parametrů modelu.

Pro počáteční odhad parametrů použijte následující střelcovy znalosti/pozorování:
- Jeho poslední vzduchovka měla úsťovou rychlost $160$ [$m/s$],
- Diabolka má ve směru letu tvar kruhu o poloměru $0.003$ [$m$]. Její plocha je tedy cca $2.8 \cdot 10^{-5}$ [$m^2$]. hustota vzduchu je cca $1.225$ [$kg/m^3$]. Tedy koeficient odporu vzduchu bude v okolí $3.5 \cdot 10^{-5}$ [$kg/m$].
- Cítí slabý vítr který mu naráží na levou tvář, takže $w_x < 0 $ a $w_y > 0$, např. $w_x = -1$ [$m/s$], $w_y = 1$ [$m/s$].

**Důležité poznámky:**
- Máte k dispozici `model_pohybu(beta, rho, gamma, alpha, wx, wy)` pro výpočet zásahu do terče v závislosti na úhlech a parametrech modelu.
- Funkce počítající chybu $\mathbf{E}$ přijimá jako jedinný argument vektor parametrů $\mathbf{p} = [\rho, \alpha, w_x, w_y]$.
- Naměřená data jsou uložena v globálních proměnných `beta`, `gamma`, `z`, `y`. Použijteje přímo takto ve funkci počítající chybu $\mathbf{E}$.
- Funkce musí vracet pouze jedno číslo a to velikost chyby $\mathbf{E}$.

In [ ]:
# Ukol 3
import scipy.optimize as opt

def E_nelinearni_ctverce(parametry):
    ######## doplnte kod #######################
    E = 0
    for i in range(4):
        z_trefa, y_trefa = model_pohybu(beta[i], gamma[i], parametry[0], parametry[1], parametry[2], parametry[3])
        E += (z_trefa - z[i])**2 + (y_trefa - y[i])**2
    ##############################################
    return E


res = opt.minimize(E_nelinearni_ctverce, [160, 3.5e-5, -1, 1],
                   method='Nelder-Mead', tol=1e-9, options={'maxiter': 10000})
parametry = res.x
print("Nalezli jsme následující hodnoty parametrů:")
print(f"rho = {parametry[0]:.2f} [m/s],")
print(f"alpha = {parametry[1]:.2e} [kg/m],")
print(f"wx = {parametry[2]:.2f} [m/s],")
print(f"wy = {parametry[3]:.2f} [m/s]")

A skutečné skryté parametry modelu jsou:

$\rho = 200$ [m/s],

$\alpha = 1.7\cdot 10^{-05}$ [kg/m],

$w_x = -5$ [m/s],

$w_y = 3$ [m/s]

---

## Poslední krok: známe model a parametry... pojďme optimalizovat!

Známe-li model včetně parametrů, můžeme si model představit jako funkci která v závislosti na vstupních úhlech $\beta$ a $\gamma$ počítá zásah do terče:
$$ \mathbf{f}(\beta, \gamma) = \begin{bmatrix} z \\ y \end{bmatrix}.$$

Chceme se trefit přesně do bodu $(0, 0)$, tedy hledáme takové $\beta$ a $\gamma$, pro které platí:
$$ \mathbf{f}(\beta, \gamma) = \begin{bmatrix} z \\ y \end{bmatrix} = \begin{bmatrix} 0 \\ 0 \end{bmatrix}.$$

Opět v duchu nejmenších čtverců budeme předpokládat, že přesné řešení nemusí existovat, spokojíme se s takovým které má nejmenší chybu:
$$ \mathbf{E}(\beta, \gamma) = \left\Vert \mathbf{f}(\beta, \gamma) - \begin{bmatrix} 0 \\ 0 \end{bmatrix} \right\Vert^2.$$

Tedy řešíme úlohu:
$$ \beta^*, \gamma^* = \arg \min_{\beta, \gamma} \mathbf{E}(\beta, \gamma).$$

Což už ale umíme. Je tedy třeba napsat funkci počítající chybu $\mathbf{E}$ a použít ji ve funkci `minimize` z knihovny `scipy.optimize`.

- Stejně jako předtím použijte globální proměnnou `parametry` s nalezenými hodnotami $\rho^*, \alpha^*, w_x^*, w_y^*$. A funkci počítající model `model_pohybu(beta, gamma, rho, alpha, wx, wy)`.

---
## Ukol 4.

Nalezněte optimální úhly pro trefení do terče.

Počáteční odhad může být například $\beta = 0$, $\gamma = 0$.

In [ ]:
# Ukol 4

def E_strelba(uhly):
    ######## doplnte kod #######################
    y, z = model_pohybu(uhly[0], uhly[1], parametry[0], parametry[1], parametry[2], parametry[3])
    sum = (y)**2 + (z)**2
    ############################################
    return sum

res = opt.minimize(E_strelba, [0, 0], method='Nelder-Mead', tol=1e-9, options={'maxiter': 10000})
optimalni_uhly = res.x

print (f"Optimalni uhly pro strelbu jsou beta = {optimalni_uhly[0]:.3f} [°], gamma = {optimalni_uhly[1]:.3f} [°]")

In [ ]:
# ověříme, kam střela dopadne
z_trefa, y_trefa = vystrel(res.x[0], res.x[1])
print(f"Střela dopadne na souřadnice z = {1000 * z_trefa:.2f} [mm], y = {1000 * y_trefa:.2f} [mm]")

**Dokonalá trefa! Doslova jen o vlas (Lidský vlas má tloušťku asi 0,05–0,1 mm) vedle středu...**

</br>
</br>
</br>
</br>
</br>
</br>

---
---
---
---
# Bonusové materiály:
---


## Mohl to náš ostrostřelec udělat už po dvou střelách?


#### Pomocí lineárního modelu 

Viděli jsme, že i takto "jednoduchý" model může být velice přesný. Co kdybychom měli jen dvě testovací střely? Šlo by to i tak spočítat?

Zkusme se nad tím zamyslet... 

Pro každou z úloh budeme mít celkem dvě měření, ale budeme muset určit 3 parametry. Možná už víte, že v takových případech může být řešení nejednoznačné.

**Tak co s tím?**

Nezbývá než zjednodušit model. Z minulých výsledků víme, že $z$ téměř výhradně závisí na $\beta$ a $y$ na $\gamma$. Můžeme tedy zkusit zjednodušený lineární model:
$$
\begin{aligned}
    z &= a_{1} \cdot \beta + a_3, \\
    y &= b_{2} \cdot \gamma + b_3.
\end{aligned}
$$

Výsledné matice budou různé: 
$$
\mathbf{M_a} \mathbf{a} = \mathbf{z}, \quad \mathbf{M_b} \mathbf{b} = \mathbf{y},
$$
kde:
$$
\mathbf{M_a} = \begin{bmatrix}
\beta_1 & 1 \\
\beta_2 & 1
\end{bmatrix}, \quad \mathbf{M_b} = \begin{bmatrix}
\gamma_1 & 1 \\
\gamma_2 & 1
\end{bmatrix}.
$$

A výsledný model bude mít pouze diagonální matici, tedy požadované úhly budou řešením soustavy rovnic:
$$ \mathbf{A} \mathbf{x} = \mathbf{f},$$
kde: 
$$
\mathbf{A} = \begin{bmatrix}
a_{1} & 0 \\
0 & b_{2}
\end{bmatrix},\quad \mathbf{x} = \begin{bmatrix} \beta \\ \gamma \end{bmatrix}, \quad \mathbf{f} = \begin{bmatrix} -a_3 \\ -b_3 \end{bmatrix}.
$$


---
## Ukol B1.
Proveďte totéž co v úkolu 1 a 2, ale s pouze s redukovaným lineárním modelem a prvními dvěma měřeními.  

In [ ]:
# Ukol 1

######## doplnte kod #######################

# sestavení matice M
N = 2
Ma = np.vstack((beta[:2], np.ones(N))).T
Mb = np.vstack((gamma[:2], np.ones(N))).T

# výpočet a
a = np.linalg.solve(Ma.T @ Ma, Ma.T @ z[:2])

# výpočet b
b = np.linalg.solve(Mb.T @ Mb, Mb.T @ y[:2])

##############################################

print('a =', a, 'b =', b)

######## doplnte kod #######################
A = np.array([[a[0], 0], [0, b[0]]])
f = np.array([-a[1], -b[1]])

x = np.linalg.solve(A, f)
##############################################
print('x =', x)

In [ ]:
# ověříme, kam střela dopadne
z_trefa, y_trefa = vystrel(x[0], x[1])
print(f"Střela dopadne na souřadnice z = {1000 * z_trefa:.2f} [mm], y = {1000 * y_trefa:.2f} [mm]")

**Stále se trefíme prakticky (méně než 1mm) přesně do středu terče!**

---

## Úkol B2

Jak přesně jsme schopni trefit terč už na druhý pokus?

Zde by už lineární model nestačil, použijte druhý přístup řešení pomocí parametrů pro fyzikální model trajektorie a pouze pro jedno měření.

In [ ]:
# Ukol B2
import scipy.optimize as opt


def E_nelinearni_ctverce(parametry):
    ######## doplnte kod #######################
    E = 0
    i = 3
    z_trefa, y_trefa = model_pohybu(beta[i], gamma[i], parametry[0], parametry[1], parametry[2], parametry[3])
    E += (z_trefa - z[i])**2 + (y_trefa - y[i])**2
    ##############################################
    return E


res = opt.minimize(E_nelinearni_ctverce, [160, 3.5e-5, -1, 1],
                   method='Nelder-Mead', tol=1e-9, options={'maxiter': 10000})
parametry = res.x

def E_strelba(uhly):
    ######## doplnte kod #######################
    y, z = model_pohybu(uhly[0], uhly[1], parametry[0], parametry[1], parametry[2], parametry[3])
    sum = (y)**2 + (z)**2
    ############################################
    return sum


res = opt.minimize(E_strelba, [0, 0], method='Nelder-Mead', tol=1e-9, options={'maxiter': 10000})
optimalni_uhly = res.x

print(f"Optimalni uhly pro strelbu jsou beta = {optimalni_uhly[0]:.3f} [°], gamma = {optimalni_uhly[1]:.3f} [°]")

# ověříme, kam střela dopadne
z_trefa, y_trefa = vystrel(res.x[0], res.x[1])
print(f"Střela dopadne na souřadnice z = {1000 * z_trefa:.2f} [mm], y = {1000 * y_trefa:.2f} [mm]")